In [1]:
import numpy as np
import pandas as pd
import string
from collections import Counter
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk
#nltk.download('vader_lexicon')
import warnings
warnings.filterwarnings('ignore')

In [3]:
df_train=pd.read_csv("D:/Flip Robo/Project_1/train.csv")

In [4]:
df_train.head()

,User_ID,Description,Browser_Used,Device_Used,Is_Response
0,id10326,The room was kind of clean but had a VERY stro...,Edge,Mobile,not happy
1,id10327,I stayed at the Crown Plaza April -- - April -...,Internet Explorer,Mobile,not happy
2,id10328,I booked this hotel through Hotwire at the low...,Mozilla,Tablet,not happy
3,id10329,Stayed here with husband and sons on the way t...,InternetExplorer,Desktop,happy
4,id10330,My girlfriends and I stayed here to celebrate ...,Edge,Tablet,not happy


In [6]:
df_test=pd.read_csv("D:/Flip Robo/Project_1/test.csv")

In [7]:
df_test.head()

,User_ID,Description,Browser_Used,Device_Used
0,id80132,Looking for a motel in close proximity to TV t...,Firefox,Mobile
1,id80133,Walking distance to Madison Square Garden and ...,InternetExplorer,Desktop
2,id80134,Visited Seattle on business. Spent - nights in...,IE,Tablet
3,id80135,This hotel location is excellent and the rooms...,Edge,Mobile
4,id80136,This hotel is awesome I love the service Antho...,Mozilla,Mobile


In [8]:
df_train.isnull().any()

User_ID         False
Description     False
Browser_Used    False
Device_Used     False
Is_Response     False
dtype: bool

In [9]:
df_test.isnull().any()

User_ID         False
Description     False
Browser_Used    False
Device_Used     False
dtype: bool

In [10]:
df_train.shape

(38932, 5)

In [11]:
df_test.shape

(29404, 4)

In [12]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38932 entries, 0 to 38931
Data columns (total 5 columns):
User_ID         38932 non-null object
Description     38932 non-null object
Browser_Used    38932 non-null object
Device_Used     38932 non-null object
Is_Response     38932 non-null object
dtypes: object(5)
memory usage: 1.5+ MB


In [13]:
df_train.describe()

,User_ID,Description,Browser_Used,Device_Used,Is_Response
count,38932,38932,38932,38932,38932
unique,38932,38932,11,3,2
top,id41904,I just got back from a week at the LAX Westin ...,Firefox,Desktop,happy
freq,1,1,7367,15026,26521


In [14]:
df_train['Is_Response'].value_counts()

happy        26521
not happy    12411
Name: Is_Response, dtype: int64

In [15]:
df_train.drop(columns = ['User_ID', 'Browser_Used', 'Device_Used'], inplace = True)

In [16]:
df_train['Is_Response'] = df_train['Is_Response'].map({'happy' : 'positive', 'not happy' : 'negative'})
df_train.sample(3)

,Description,Is_Response
31127,"Good hotel, great tourist location! Near all F...",positive
35071,Not what I usually expect from DoubleTree. The...,negative
22042,I just returned from a stay at the Joule and f...,negative


In [17]:
import re
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer

tokenizer = WordPunctTokenizer()
twitter_handle = r'@[A-Za-z0-9_]+'                         # remove twitter handle (@username)
url_handle = r'http[^ ]+'                                  # remove website URLs that start with 'https?://'
combined_handle = r'|'.join((twitter_handle, url_handle))  # join
www_handle = r'www.[^ ]+'                                  # remove website URLs that start with 'www.'
punctuation_handle = r'\W+'

In [18]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [19]:
def process_text(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()

    try:
        text = souped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        text = souped

    cleaned_text = re.sub(punctuation_handle, " ",(re.sub(www_handle, '', re.sub(combined_handle, '', text)).lower()))
    cleaned_text = ' '.join([word for word in cleaned_text.split() if word not in stopwords.words()])

    return (" ".join([word for word in tokenizer.tokenize(cleaned_text) if len(word) > 1])).strip()

In [20]:
cleaned_text = []

for text in df_train.Description:
    cleaned_text.append(process_text(text))

clean_text = pd.DataFrame({'clean_text' : cleaned_text})
df_train = pd.concat([df_train, clean_text], axis = 1)

df_train.sample(3)

,Description,Is_Response,clean_text
17012,I was a litle concerned about staying at the I...,positive,litle concerned staying ballpark reading revie...
17779,"This hotel is convenient to IND airport, which...",negative,convenient airport great flight arriving late ...
22355,"I go to NY every year for a """"""""""""""""""""""""""""""""""""...",positive,go ny every year girls weekend stayed several ...


In [22]:
from sklearn.model_selection import train_test_split

attribute = df_train.clean_text
target = df_train.Is_Response

In [23]:
attribute_train, attribute_test, target_train, target_test = train_test_split(attribute, target, test_size = 0.1, random_state = 225)

print('attribute_train :', len(attribute_train))
print('attribute_test  :', len(attribute_test))
print('target_train :', len(target_train))
print('target_test  :', len(target_test))

attribute_train : 35038
attribute_test  : 3894
target_train : 35038
target_test  : 3894


In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

tvec = TfidfVectorizer()
clf2 = LogisticRegression()

In [25]:
from sklearn.pipeline import Pipeline

model = Pipeline([('vectorizer',tvec)
                 ,('classifier',clf2)])

model.fit(attribute_train, target_train)

Pipeline(memory=None,
     steps=[('vectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=...penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))])

In [26]:
example_text = ["I'm very happy now"]
example_result = model.predict(example_text)

print(example_result)

['positive']


In [28]:
from sklearn.metrics import confusion_matrix

verdict = model.predict(attribute_test)

confusion_matrix(verdict, target_test)

array([[1001,  153],
       [ 322, 2418]], dtype=int64)

In [29]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

print("Accuracy : ", accuracy_score(verdict, target_test))
print("Precision : ", precision_score(verdict, target_test, average = 'weighted'))
print("Recall : ", recall_score(verdict, target_test, average = 'weighted'))

Accuracy :  0.8780174627632255
Precision :  0.8859977141592815
Recall :  0.8780174627632255
